# Architectures

> Scripts to get architectures for the models

In [22]:
#| default_exp architectures

In [23]:
#| export
#| hide

import torch
import torch.nn as nn
from abc import ABC, abstractmethod

In [24]:
#| export
class Sampling(nn.Module):
    """
    PyTorch module to perform the reparameterization trick for VAEs.
    """
    def forward(self, z_mean, z_log_var):
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

In [25]:
#| export
class Architecture(ABC):
    """
    Abstract base class for all architectures. Requires subclasses to define a type.
    """
    @property
    @abstractmethod
    def arch_type(self):
        """
        Returns the architectural type of the model.
        Subclasses must provide this property.
        """
        pass

In [26]:
#| export
class BaseVAEArchitecture(Architecture):
    """
    Abstract base class for Variational Autoencoder architectures.
    """
    def __init__(self):
        super().__init__()
        self._encoder = None
        self._decoder = None

    @property
    def encoder(self):
        """
        Property for accessing the encoder model.
        Raises NotImplementedError if the encoder model is not implemented.
        """
        if self._encoder is None:
            raise NotImplementedError("Encoder model has not been implemented.")
        return self._encoder

    @property
    def decoder(self):
        """
        Property for accessing the decoder model.
        Raises NotImplementedError if the decoder model is not implemented.
        """
        if self._decoder is None:
            raise NotImplementedError("Decoder model has not been implemented.")
        return self._decoder

    def get(self):
        """
        Retrieves both encoder and decoder models as a dictionary.
        Raises NotImplementedError if either model is not implemented.
        """
        if self._encoder is None or self._decoder is None:
            raise NotImplementedError("Encoder or Decoder model has not been fully implemented.")
        return {"encoder": self._encoder, "decoder": self._decoder}

In [27]:
#| export
class VAE_CONV5Architecture(BaseVAEArchitecture):
    """
    This class defines the architecture for a Variational Autoencoder (VAE) with Convolutional Layers.
    Parameters:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of latent space.
        dropout_rate (float): Dropout rate to use in dropout layers.
    """
    @property
    def arch_type(self):
        return "vae_conv5"

    def __init__(self, seq_len, feat_dim, latent_dim, dropout_rate=0.2):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        self._encoder = self._build_encoder()
        self._decoder = self._build_decoder()

    def _build_encoder(self):
        # Group 1: Transpose Convolution Layers to reshape and refine the spatial structure
        convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Flatten()
        )
        # Group 2: Dense Layers to downscale the feature space to a lower latent dimensions
        dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )
        return nn.Sequential(convo_layers, dense_layers, Sampling())
    
    def _build_decoder(self):
        # Group 1: Dense Layers to upscale the latent dimensions to a higher feature space
        dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, self.seq_len))
        )

        # Group 2: Transpose Convolution Layers to reshape and refine the spatial structure
        transpose_conv_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=3, stride=1, padding='same'),
            nn.Sigmoid()
        )

        # Combine both groups into the final sequential model
        return nn.Sequential(dense_layers, transpose_conv_layers)

In [28]:
#| hide
import nbdev; nbdev.nbdev_export()

/usr/local/lib/python3.10/dist-packages/nbdev/export.py:80: UserWarning: Notebook '/orbit-generation/nbs/10_vae.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
